In [29]:
# -*- coding: utf-8 -*-

# **Jupyter Notebook: AI-помощник на базе LLM с RAG для запросов к базе знаний**

## **1. Введение**

### Данный ноутбук посвящен разработке интеллектуального помощника, использующего большие языковые модели (LLM) и Retrieval-Augmented Generation (RAG) для ответов на вопросы на основе предоставленной базы знаний.

### **Бизнес-проблема:** Компании часто имеют обширную внутреннюю документацию, к которой трудно получить быстрый и точный доступ. AI-помощник с RAG может предоставлять контекстно-зависимые ответы, улучшая эффективность и удовлетворенность.

### **Цель проекта:** Построить RAG-систему, способную отвечать на вопросы на основе реального текстового корпуса, используя бесплатные ресурсы Google Colab.

In [2]:
# ## 2. установка и импорт необходимых библиотек, настройка среды
# Здесь мы импортируем все библиотеки, которые потребуются для проекта.
# Если ноутбук запускается в Google Colab, убедитесь, что используется среду с GPU (T4).

!pip install transformers sentence-transformers langchain langchain-community faiss-cpu accelerate bitsandbytes -q
!pip install pypdf -q # Для чтения PDF, если понадобится


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import requests # Для загрузки данных
from tqdm.notebook import tqdm # Для прогресс-баров
from google.colab import userdata
import os

# Для RAG
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
from langchain_core.documents import Document

In [4]:
# Проверяем доступность GPU
if torch.cuda.is_available():
    print(f"GPU доступен: {torch.cuda.get_device_name(0)}")
    device = "cuda"
else:
    print("GPU не доступен, будет использоваться CPU.")
    device = "cpu"


GPU доступен: Tesla T4


### Для начала собираем данные для создаваемой нами базы знаний, которую мы сформируем из англоязычного текста Библии короля Якова (KJB), опубликованного на сайте проекта Project Gutenberg. Это большой корпус, состаящий из 39 книг, и он хорошо подходит в качестве коллекции документов различающихся стилем и манерой повествования. После этого производится очистка (предполагает удаление метаданных)

### Источник данных: "The King James Version of the Bible" (Библия короля Якова)
### URL: https://www.gutenberg.org/files/10/10-0.txt

In [6]:
print("Загрузка текста книги из Project Gutenberg...")
book_url = "https://www.gutenberg.org/files/10/10-0.txt"
response = requests.get(book_url)
response.raise_for_status() # Проверка, успешности запроса

raw_text = response.text
print(f"Текст успешно загружен. Размер текста: {len(raw_text)} символов.")

# Очистка текста: удаляем метаданные Project Gutenberg в начале и конце
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK 10 ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK 10 ***"

start_index = raw_text.find(start_marker)
end_index = raw_text.find(end_marker)

if start_index!= -1 and end_index!= -1:
    cleaned_text = raw_text[start_index + len(start_marker):end_index].strip()
    print(f"Текст очищен от метаданных. Размер очищенного текста: {len(cleaned_text)} символов.")
else:
    cleaned_text = raw_text.strip()
    print("Метаданные не найдены, используется весь текст.")

# Сохраняем очищенный текст в файл для дальнейшей обработки
with open("KJB.txt", "w", encoding="utf-8") as f:
    f.write(cleaned_text)
print("Очищенный текст сохранен в 'KJB.txt'.")

Загрузка текста книги из Project Gutenberg...
Текст успешно загружен. Размер текста: 4432276 символов.
Текст очищен от метаданных. Размер очищенного текста: 4432176 символов.
Очищенный текст сохранен в 'KJB.txt'.


## 4. Предварительная обработка текста и генерация эмбеддингов

## На этом этапе мы подготовим текстовые данные для RAG-системы:
## - **Разбиение на чанки (Chunking):**
## - **Генерация эмбеддингов:**
## - **Индексация эмбеддингов в векторной базе данных FAISS**

In [7]:
from langchain_core.documents import Document

# Обёртывание текста в объект Document
documents = [Document(page_content=cleaned_text, metadata={"source": "your_source"})]

# Разбиение
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)
chunks = text_splitter.split_documents(documents)  # Список чанков

print(f"Текст разбит на {len(chunks)} чанков.")
print("Пример чанка:")
print(chunks[25].page_content[:1000] + "...")

Текст разбит на 11137 чанков.
Пример чанка:
unto the serpent, We may eat of the fruit of the trees of the garden:
3:3 But of the fruit of the tree which is in the midst of the garden,
God hath said, Ye shall not eat of it, neither shall ye touch it, lest
ye die.

3:4 And the serpent said unto the woman, Ye shall not surely die: 3:5
For God doth know that in the day ye eat thereof, then your eyes shall
be opened, and ye shall be as gods, knowing good and evil.

3:6 And when the woman saw that the tree was good for food, and that...


### Создаём из чанков эмбединги с использованием небольшой и эффективной модели семантического поиска **all-MiniLM-L6-v2** и индексируем в базе данных FAISS, которая позволяет нам эффективно искать сходство между векторами.

In [8]:
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs={'device': device})

/tmp/ipython-input-8-3813106297.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs={'device': device})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# 4.3. Индексация в векторной базе данных (FAISS)
vector_store = FAISS.from_documents(chunks, embeddings)

# Сохраняем векторную базу данных локально (для использования в API)
vector_store_path = "/tmp/faiss_index_KJB"
vector_store.save_local(vector_store_path)
print(f"Путь до векторной базы данных FAISS: {vector_store_path}")

Путь до векторной базы данных FAISS: /tmp/faiss_index_KJB


## 5. Настройка Большой Языковой Модели (LLM)

### Генерировать ответы мы будем с помощью небольшой и мощной модели **gemma-2b-it**, работающей даже на CPU применяя 4-битную квантизацию, чтобы уместить модель в память GPU и ускорить её.

In [10]:
# Конфигурация для 4-битной квантизации
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Загрузка токенизатора и модели
llm_model_name = "google/gemma-2b-it"
print(f"Загрузка токенизатора для {llm_model_name}...")

Загрузка токенизатора для google/gemma-2b-it...


### Модель 'google/gemma-2b-it' является "gated repo" на Hugging Face. Для доступа к ней необходимо:
1. Перейти на страницу модели на Hugging Face (https://huggingface.co/google/gemma-2b-it) и принять условия использования.
2. Сгенерировать API токен в настройках Hugging Face (https://huggingface.co/settings/tokens).
3. Добавить этот токен в секреты Colab с именем HF_TOKEN.
4. Перезапустить сессию Colab (Runtime -> Restart session).

In [11]:
# # Загрузка токена из секретов Colab и установка его как переменной окружения для аутентификации при загрузке gated моделей с Hugging Face.
# from google.colab import userdata
# import os

# hf_token = userdata.get('HF_TOKEN')
# if hf_token:
#     os.environ["HF_TOKEN"] = hf_token
#     print("Hugging Face token loaded from Colab secrets.")
#     if "HF_TOKEN" in os.environ:
#         print("HF_TOKEN environment variable is set.")
# else:
#     print("Hugging Face token not found in Colab secrets. Please add it as HF_TOKEN.")
#     print("HF_TOKEN environment variable is NOT set.")


# # tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
# # print(f"Загрузка модели {llm_model_name} с 4-битной квантизацией...")
# # model_llm = AutoModelForCausalLM.from_pretrained(
# #     llm_model_name,
# #     quantization_config=nf4_config,
# #     torch_dtype=torch.bfloat16,
# #     device_map="auto" # Автоматически распределяет модель по доступным устройствам (GPU/CPU)
# # )
# # print("Модель LLM загружена.")

# # Создание HuggingFace Pipeline для генерации текста
# # Pass the model name string to the pipeline
# text_generation_pipeline = pipeline(
#     task="text-generation", # Explicitly specify the task
#     model=llm_model_name, # Pass the model name string
#     # tokenizer=tokenizer, # Remove tokenizer argument when passing model name
#     max_new_tokens=256, # Максимальное количество новых токенов в ответе
#     do_sample=True,
#     temperature=0.7,
#     top_k=50,
#     top_p=0.95,
#     repetition_penalty=1.1, # Для предотвращения повторений
#     model_kwargs={"quantization_config": nf4_config, "torch_dtype": torch.bfloat16} # Pass quantization config via model_kwargs
# )
# print("HuggingFace Pipeline для генерации текста настроен.")


# # Оборачиваем пайплайн в LangChain LLM
# llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
# print("LLM-пайплайн для LangChain настроен.")

### Загружаем квантизировагнную модель LLM на доступное устройство, далее создаём пайплайн HuggingFace для генерации текста ответа.

In [12]:
# Загружаем модель с Hugging Face.
from google.colab import userdata
import os

hf_token = userdata.get('HF_TOKEN')
if hf_token:
    os.environ["HF_TOKEN"] = hf_token
else:
    print("Токен Hugging Face не найден")

tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
print(f"Загрузка модели {llm_model_name} с 4-битной квантизацией...")
model_llm = AutoModelForCausalLM.from_pretrained(
    llm_model_name,
    quantization_config=nf4_config,
    torch_dtype=torch.bfloat16,
    device_map="auto") # Автозагрузка модели на доступное устройство (GPU/CPU)

# Создание пайплайна
text_generation_pipeline = pipeline(
    task="text-generation",
    model=llm_model_name,
    # tokenizer=tokenizer, # Remove tokenizer argument when passing model name
    max_new_tokens=64, # Максимальное количество новых токенов в ответе
    do_sample=True,
    temperature=0.3,
    top_k=75,
    top_p=0.97,
    repetition_penalty=1.1, # Для предотвращения повторений
    model_kwargs={"quantization_config": nf4_config, "torch_dtype": torch.bfloat16} # передача конфигурации квантизации
)
print("HuggingFace Pipeline для генерации текста настроен.")


# Оборачиваем пайплайн в LangChain LLM
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
print("LLM-пайплайн для LangChain настроен.")


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Загрузка модели google/gemma-2b-it с 4-битной квантизацией...


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


HuggingFace Pipeline для генерации текста настроен.
LLM-пайплайн для LangChain настроен.


/tmp/ipython-input-12-1867186861.py:36: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


### Конвейер RAG остоит из двух составляющих:

### 1.  **Retrieval (Получение):** Извлечение на основе запроса пользователя релевантных текстовых фрагментов из базы знаний.
### 2.  **Generation (Генерация):** Использование LLM для создания ответа, основываясь на извлеченном контексте.

### Создаём промпт, инструктирующий LLM как ей необходимо отвечать на вопросы исходя из контекста, а также RAG-цепочку, связывающую векторный поиск с генерацией,  а также ретривер для извлечения релевантного контента из векторной базы данных (задаётся количество выдаваемых единиц контента).


# Порядок операций в RAG-цепочке
### 1. Получение вопроса
### 2. Передаем вопроса ретриверу для извлечения контекста
### 3. Формирование промпта с контекстом и вопросом
### 4. Передача промпта в LLM для генерации ответа
### 5. Парсинг выводимого LLM текста в строку

In [ ]:
# Промпт
template = """Используй следующий контекст, чтобы ответить на вопрос пользователя.
Если ты не знаешь ответа, просто скажи, что не можешь найти информацию в предоставленном контексте.
Не придумывай ответ если не знаешь.

Контекст:
{context}

Вопрос: {question}

Ответ:"""

RAG_PROMPT_TEMPLATE = PromptTemplate.from_template(template)

# Создание RAG-цепочки (LangChain)
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Создаем ретривер из векторного хранилища
retriever = vector_store.as_retriever(search_kwargs={"k": 23})

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}

| RAG_PROMPT_TEMPLATE
| llm
| StrOutputParser()
)

print("Конвейер RAG успешно настроен.")

### Мы построили RAG систему. Самое время исппытать её и посмотреть что у нас получилось.

In [ ]:
# --- 7. Тестирование RAG-системы ---

print("\n--- Тестирование RAG-системы ---")

# Вопрос 1: Вопрос, на который есть ответ в тексте
question1 = "Why James decided to runaway?"
print(f"Вопрос: {question1}")
response1 = rag_chain.invoke(question1)
print(f"Ответ: {response1}")
print("-" * 50)

# Вопрос 2: Вопрос о персонаже
question2 = "What were the names of Adam's' sons?"
print(f"Вопрос: {question2}")
response2 = rag_chain.invoke(question2)
print(f"Ответ: {response2}")
print("-" * 50)

# # Вопрос 3: Вопрос, на который, возможно, нет прямого ответа в контексте
question3 = "Who is Moses?"
print(f"Вопрос: {question3}")
response3 = rag_chain.invoke(question3)
print(f"Ответ: {response3}")
print("-" * 50)

# # Вопрос 4: Вопрос о деталях сюжета
question4 = "Who was the serpent?"
print(f"Вопрос: {question4}")
response4 = rag_chain.invoke(question4)
print(f"Ответ: {response4}")
print("-" * 50)


In [ ]:

# Пример 5: Вопрос о деталях сюжета
question5 = "What was Nebuchadnezzar?"
print(f"Вопрос: {question5}")
response5 = rag_chain.invoke(question5)
print(f"Ответ: {response5}")
print("-" * 50)


In [ ]:

# Пример 5: Вопрос о деталях сюжета
question5 = "What was a name of Adam's wife"
print(f"Вопрос: {question5}")
response5 = rag_chain.invoke(question5)
print(f"Ответ: {response5}")
print("-" * 50)


In [44]:

print("\nВывод по тестированию RAG-системы:")
print(" - Система успешно извлекает релевантный контекст и генерирует ответы.")
print(" - Для вопросов вне контекста базы знаний, LLM должна отвечать, что не может найти информацию.")

#
# ## 8. Оценка и оптимизация RAG-системы (концептуально)
#
# Оценка RAG-систем — это сложная задача, требующая как традиционных метрик NLP,
# так и метрик, специфичных для качества генерации и получения информации.
#
# **Метрики оценки (концептуально):**
# -   **Relevance (Релевантность):** Насколько ответ релевантен запросу.
# -   **Faithfulness (Достоверность):** Насколько ответ основан на предоставленном контексте.
# -   **Context Precision/Recall (Точность/Полнота контекста):** Насколько извлеченный контекст
#     был релевантен и полон.
# -   **Answer Correctness (Правильность ответа):** Насколько ответ верен фактически.
# -   **Fluency (Беглость), Coherence (Связность).**
#
# **Инструменты:** RAGAS (Rag-as-a-Service), TruLens, LangChain Evaluators (требуют размеченных данных).
#
# **Оптимизация:** Экспериментирование с:
# -   Различными стратегиями разбиения на чанки (размер, перекрытие).
# -   Моделями эмбеддингов.
# -   Порогами поиска в векторной базе данных (параметр `k` в ретривере).
# -   Шаблонами промптов для LLM.
# -   Моделями LLM (использование более крупных или специализированных моделей).
#
# Для полноценной оценки требуются размеченные пары "вопрос-ответ" с указанием релевантных фрагментов текста.
# В рамках этого ноутбука мы проводим только качественную оценку на примерах.

print("\n--- Концепция оценки и оптимизации конвейера RAG ---")
print("Для полноценной оценки RAG-системы требуются размеченные данные (вопросы, правильные ответы, релевантные чанки).")
print("Оптимизация включает эксперименты с параметрами разбиения на чанки, моделями эмбеддингов, LLM и промптами.")

#
# ## 9. Сохранение артефактов для развертывания
#
# Для развертывания RAG-системы в виде сервиса нам нужно сохранить обученные компоненты:
# -   Векторную базу данных FAISS.
# -   Название модели эмбеддингов (для загрузки в сервисе).
# -   Название LLM (для загрузки в сервисе).
# -   Шаблон промпта.

# Векторная база данных уже сохранена в разделе 4: /tmp/faiss_index_alice
# model_llm и tokenizer будут загружены по имени в сервисе.
# embeddings также будут загружены по имени.

# Сохраняем информацию о моделях для сервиса
rag_config = {
    "embedding_model_name": embedding_model_name,
    "llm_model_name": llm_model_name,
    "vector_store_path": vector_store_path,
    "prompt_template": template
}

import json
with open("/tmp/rag_config.json", "w") as f:
    json.dump(rag_config, f, indent=4)
print("Конфигурация RAG-системы сохранена в '/tmp/rag_config.json'.")

print("\nВывод по сохранению артефактов:")
print(" - Векторная база данных FAISS и конфигурация RAG-системы сохранены.")
print(" - Эти файлы будут использоваться для инициализации RAG-системы в FastAPI, Gradio и Streamlit сервисах.")


Вывод по тестированию RAG-системы:
 - Система успешно извлекает релевантный контекст и генерирует ответы.
 - Для вопросов вне контекста базы знаний, LLM должна отвечать, что не может найти информацию.

--- Концепция оценки и оптимизации конвейера RAG ---
Для полноценной оценки RAG-системы требуются размеченные данные (вопросы, правильные ответы, релевантные чанки).
Оптимизация включает эксперименты с параметрами разбиения на чанки, моделями эмбеддингов, LLM и промптами.
Конфигурация RAG-системы сохранена в '/tmp/rag_config.json'.

Вывод по сохранению артефактов:
 - Векторная база данных FAISS и конфигурация RAG-системы сохранены.
 - Эти файлы будут использоваться для инициализации RAG-системы в FastAPI, Gradio и Streamlit сервисах.
